# Лекция 3 "Data types"
__План:__
  - Custom types (Type aliases)
  - Алгебраические типы данных
    - Произведение-тип
    - Сумма-тип
    - Примеры
  - Record syntax
    - Sample
    - Operator record fields
    - Records и Сумма-типы
    - Duplicate record fields
    - Record wild cards
  - _newtype_
  - Type classes
    - _instance_
    - class ```Eq```
    - class ```Ord```
    - class ```Num```
    - class ```Show```
    - class ```Read```
    - Addition*
  - ```deriving```
  

## Type aliases

Это простое переобозначение типов. Используется для улучшения читабельности кода.  
* Без использования _type aliases_:  
```userFullId :: (Int, String, String) -> String```  
```userFullId (uid, login, name) = show uid ++ ":" ++ login```  

* Используя _type aliases_:  
```type User = (Int, String, String)```  
```userFullId :: User -> String```  
```userFullId (uid, login, name) = show uid ++ ":" ++ login```

__Но__ при использовании все равно в конструкторе указываем поля, соответствующие типам, что не избавляет нас от проблем с патернматчингом. 


## ADT (Алгебраические типы данных)

### Произведение-тип

$$
PT = T_1 \times T_2 \times \cdots \times T_n
$$
Пример:
$$
user = int \times string \times string
$$

__C++__  
```
struct user {
    int uid;
    string login;
    string pass;
};
```
__Haskell__
```
type User = (Int, String, String)
```

### Сумма-тип

$$
ST = T_1 + T_2 + \cdots + T_n
$$

$$
PrimitiveType::=Int ∣ Char ∣ Double ∣ \cdots
$$

$$
ADT::=PrimitiveType ∣ ADT + ADT ∣ ADT \times ADT
$$
Пример:  

__Haskell__  
```
data TrafficLight = Red | Yellow | Green | Blue
```
_Замечание:_ ```Red```, ```Yellow```, ... это __конструкторы__


### Примеры на __Haskell__

```
      ┌─ type name
      │
      │       ┌─ constructor name (or constructor tag)
      │       │
data User = MkUser Int String String
│                  │    │      │
│                  └────┴──────┴── types of fields
│
└ "data" keyword

ghci> :t MkUser
MkUser :: Int -> String -> String -> User
```
* __Параметризованные ADT:__

```
data Point2D a = Point2D a a  -- constructor name can be the same as type name  

pointToList :: Point2D a -> [a]
pointToList (Point2D x y) = [x, y]  

maxCoord :: Point2D Int -> Int
maxCoord (Point2D x y) = max x y

distFromZero :: Point2D Double -> Double
distFromZero (Point2D x y) = sqrt (x^2 + y^2)

ghci> :t Point2D  -- remeber, constructors are just functions
Point2D :: a -> a -> Point2D a
```
* __Сумма-тип:__

```
data IntResult = Success Int 
               | Failure String
  
ghci> :t Success 
Success :: Int -> IntResult
ghci> :t Failure 
Failure :: String -> IntResult

safeDiv :: Int -> Int -> IntResult
safeDiv _ 0 = Failure "division by zero"
safeDiv x y = Success $ x `div` y

showResult :: IntResult -> String
showResult (Success n) = "Result: " ++ show n
showResult (Failure e) = "Error:  " ++ e

ghci> showResult $ safeDiv 7 2
"Result: 3"
ghci> showResult $ safeDiv 7 0
"Error: division by zero"
```

* __Something special:__

```
data Maybe a = Nothing | Just a  -- implemented in Prelude

maybeSecond :: [a] -> Maybe a
maybeSecond (_:x:_) = Just x
maybeSecond _       = Nothing

data Either a b = Left a | Right b  -- implemented in Prelude

eitherSecond :: [a] -> Either String a
eitherSecond []      = Left "list is empty"
eitherSecond [_]     = Left "list has only single element"
eitherSecond (_:x:_) = Right x

data List a = Nil | Cons a (List a) -- recursive

ghci> :t Nil
Nil :: List a
ghci> :t Cons
Cons :: a -> List a -> List a

-- Real List --
data [] a = [] | a : [a]
```


## Record syntax

### Sample

__Record definition...__

```
data User = User 
    { uid      :: Int
    , login    :: String
    , password :: String 
    }
```
__просто синтаксический сахар для вот такой записи:__

```
data User = User Int String String

uid :: User -> Int
uid (User i _ _) = i

login :: User -> String
login (User _ l _) = l

password :: User -> String
password (User _ _ p) = p
```

* _Пример использования:_

```
ivan :: User
ivan = User { login    = "Ivan"
            , password = "123" 
            , uid      = 1
            }

isIvan :: User -> Bool
isIvan user = login user == "Ivan"

isIvan :: User -> Bool
isIvan User{ login = userName } = userName == "Ivan"

isIvan :: User -> Bool
isIvan User{ login = "Ivan" } = True
isIvan _                      = False

-- Update --
cloneIvan :: User
cloneIvan = ivan { uid = 2 }  -- User 2 "Ivan" "123"
```

### Operator record fields

В виде полей можно также перегружать __операторы__:

```
ghci> data R = R { (-->) :: Int -> Int }
ghci> let r  = R { (-->) = (+1) }
ghci> r --> 8
9

```

### Records и Сумма-типы

```
data Person 
    = User  { uid :: Int, login :: String } 
    | Admin { aid :: Int, login :: String }

login :: Person -> String  -- after desugaring
login (User  _ l) = l
login (Admin _ l) = l

isAdmin :: Person -> Bool  -- To match just the type of the construction
isAdmin Admin{} = True     -- works even without records
isAdmin _       = False
```

__Но__ надо быть очень внимательным, потому что такие записи небезопасны:

```
ghci> uid $ Admin 0 "Vasya" 
*** Exception: No match in record selector uid
```


### Duplicate record fields

```
data Man = Man { name :: String }
data Cat = Cat { name :: String }
```
Что из себя представляет ```name::```?

__Решение:__ 

* Использовать префиксы в виде названий типов (_Рекомендуется_):

```
data Man = Man { manName :: String }
data Cat = Cat { catName :: String }
```

* Использовать расштрение ```-XDuplicateRecordFields```:

```
{-# LANGUAGE DuplicateRecordFields #-}

data Man = Man { name :: String }
data Cat = Cat { name :: String }

shoutOnHumanBeing :: Man -> String
shoutOnHumanBeing man = (name :: Man -> String) man ++ "!!1!"  -- though...

isGrumpy :: Cat -> Bool
isGrumpy Cat{ name = "Grumpy" } = True
isGrumpy _                      = False
```


### Record wild cards

Хотя поля это функции, мы всё равно можем обращаться к ним, если используем расширение ```RecordWildCards```:

```
{-# LANGUAGE RecordWildCards #-}

data User = User 
    { uid      :: Int
    , login    :: String
    , password :: String 
    } deriving (Show)

toUnsafeString :: User -> String
toUnsafeString User{ uid = 0, .. } = "ROOT: " ++ login ++ ", " ++ password
toUnsafeString User{..}            = login ++ ":" ++ password
```

Также есть некоторая магия, __но__ только если поля у обоих типов __одинаковые__:

```
evilMagic :: Man -> Cat
evilMagic Man{..} = Cat{..}

ghci> evilMagic $ Man "Grumpy"
Cat {name = "Grumpy"}
```


## ___newtype___

_def:_ If __data__ type has _only one_ __constructor__ with _only one_ __field__ then it can be defined as __newtype__, which has more _efficient_ runtime representation.

```
newtype PublicKey = PublicKey String
newtype SecretKey = SecretKey String

derivePublicKey :: SecretKey -> PublicKey

checkKeyPair :: (SecretKey, PublicKey) -> Bool
checkKeyPair (secretKey, publicKey) = publicKey == derivePublicKey secretKey
```

По сути, это простое переименование какого-то типа, но с конструктором.

## Type classes

- Все классы _полиморфны_
- Класс это что-то типо _интерфейсов_

```
class Printable p where  -- we don't care what 'p' stores internally
    printMe :: p -> String
```

### _instance_

Connection between __data__ and __class__ — _```instance```_ keyword

```
instance Printable Foo where
    printMe Foo = "Foo"
    printMe Bar = "Bar (whatever)"
```

__Пример:__

```
helloP :: Printable p => p -> String
helloP p = "Hello, " ++ printMe p ++ "!"

ghci> helloP Bar
"Hello, Bar (whatever)!"
ghci> helloP True
    • No instance for (Printable Bool) arising from a use of ‘helloP’
    • In the expression: helloP True
```

Мы не прописали ```instance Printable``` для ```Bool```, что и привело к ошибке.

### class ```Eq```

```
class Eq a where  
    (==) :: a -> a -> Bool  
    (/=) :: a -> a -> Bool
 
    x == y = not (x /= y)  
    x /= y = not (x == y)
    {-# MINIMAL (==) | (/=) #-}  -- minimal complete definition
```

Заметим, что мы явно (_```{-# MINIMAL ...#-}```_) указали __минимальный__ объем функций для использования _```instance```_. Поэтому при испоьзовании ограничимся только одним из операторов:

```
{-# LANGUAGE InstanceSigs #-}

data TrafficLight = Red | Yellow | Green

instance Eq TrafficLight where
    (==) :: TrafficLight -> TrafficLight -> Bool
    Red    == Red    = True  
    Green  == Green  = True  
    Yellow == Yellow = True 
         _ == _      = False

threeSame :: Eq a => a -> a -> a -> Bool
threeSame x y z = x == y && y == z

ghci> threeSame Red Red Red
True
ghci> threeSame 'a' 'b' 'b'
False
```

Так же видно, что _символы_ поддерживают ```Eq```, потому что для них уже определена огромная куча базовых _```instance```_-ов. 

### class ```Ord```

```
data Ordering = LT | EQ | GT

-- simplified version of Ord class
class Eq a => Ord a where
   compare              :: a -> a -> Ordering
   (<), (<=), (>=), (>) :: a -> a -> Bool

   compare x y
        | x == y    =  EQ
        | x <= y    =  LT
        | otherwise =  GT

   x <= y           =  compare x y /= GT
   x <  y           =  compare x y == LT
   x >= y           =  compare x y /= LT
   x >  y           =  compare x y == GT
```

Заметим, что для задания порядка нужно обладать _```instance```_-ом от класса ```Eq```. Так же здесь не указан минимальный набор функций для достаточного определения _```instance```_-а, но в [официальной документации](https://hackage.haskell.org/package/base-4.16.0.0/docs/Data-Ord.html) написано:

```
  {-# MINIMAL compare | (<=) #-}  -- minimal complete definition
```

### class ```Num```

```
-- | Basic numeric class.
class Num a where
    {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}

    (+), (-), (*)       :: a -> a -> a  -- self-explained
    negate              :: a -> a       -- unary negation
    abs                 :: a -> a       -- absolute value
    signum              :: a -> a       -- sign of number, abs x * signum x == x
    fromInteger         :: Integer -> a -- used for numeral literals polymorphism

    x - y               = x + negate y
    negate x            = 0 - x
```

Когда мы печатаем '7' - это всего лишь синтаксический сахар для ```fromInteger 7```. Вот поэтому числовые константы __полиморфны__.  
Пример:

```
ghci> :t 5
5 :: Num p => p
ghci> :t fromInteger 5
fromInteger 5 :: Num a => a
ghci> 5 :: Int
5
ghci> 5 :: Double
5.0
```

### class ```Show```

Используется, чтобы конвертировать объект в строку.

```
-- simplified version; used for converting things into String
class Show a where
    show :: a -> String
```

```Show``` используется, чтобы вывести значение в GHCi.

```
ghci> 5
5
ghci> show 5
"5"
ghci> "5"
"5"
ghci> 5 :: Int
5
ghci> 5 :: Double
5.0
ghci> 5 :: Rational
5 % 1
```

```show``` неадекватно работает со строками: он оборачивает их в ещё одни ковычки ```"..."```

```
ghci> show "5"
"\"5\""
```

### class ```Read```

Используется для парсинга из строки в datatype.

```
-- simplified version; used for parsing thigs from String
class Read a where
    read :: String -> a

ghci> :t read
read :: Read a => String -> a
```

При этом иногда не совсем понятно, что мы хотим распарсить, в следствие чего получаем ошибку компиляции:

```
ghci> read "True"
*** Exception: Prelude.read: no parse
```

Явно укажем, что хотим получить тип _```Bool```_:

```
ghci> read "True" :: Bool
True
```

_```read```_ бросает ошибки, поэтому существуют функции для безопасного чтения (__readMaybe / readEither__) из модуля _```:module Text.Read```_:

```
ghci> :module Text.Read

ghci> :t readMaybe
readMaybe :: Read a => String -> Maybe a
ghci> :t readEither 
readEither :: Read a => String -> Either String a
ghci> readMaybe "5" :: Maybe Int
Just 5
ghci> readMaybe "5" :: Maybe Bool
Nothing
ghci> readEither "5" :: Either String Bool -- don't worry, convenient way exist
Left "Prelude.read: no parse"
```

### Addition

GHCi может помочь с определением _```instance```_-ов:

```
ghci> cmpSum x y = if x < y then x + y else x * y
ghci> :t cmpSum
cmpSum :: (Ord a, Num a) => a -> a -> a
```

__Undefined__

Используется как "заглушка" в программе:

```
foo :: (Ord a, Read a, Show b) => String -> a -> b -> b -> String
foo = undefined -- too difficult to implement
```
При попытке исполнить _```undefined```_ вылетит ошибка, __но__ только при попытке __вычислить__ его, так как наличие его __не ломает программы__ в целом.

```
ghci> :t undefined
undefined :: a
ghci> undefined
*** Exception: Prelude.undefined
CallStack (from HasCallStack):
  error, called at libraries/base/GHC/Err.hs:79:14 in base:GHC.Err
  undefined, called at <interactive>:44:1 in interactive:Ghci27
```

## ```deriving```

Автомотическое описание _```instance```_-ов:

```
data TrafficLight = Red | Yellow | Green | Blue
    deriving (Eq, Ord, Enum, Bounded, Show, Read, Ix)

ghci> show Blue
"Blue"
ghci> read "Blue" :: TrafficLight 
Blue

ghci> Red == Yellow  -- (==) is from Eq  class
False
ghci> Red < Yellow   -- (<)  is from Ord class
True

ghci> :t fromEnum 
fromEnum :: Enum a => a -> Int
ghci> :t toEnum 
toEnum :: Enum a => Int -> a
ghci> fromEnum Green
2
ghci> toEnum 2 :: TrafficLight 
Green

ghci> :t maxBound 
maxBound :: Bounded a => a
ghci> maxBound :: TrafficLight  -- Bounded also has 'minBound'
Blue
ghci> [Yello .. maxBound]  -- .. is from Enum instance
[Yellow, Green, Blue] 
```

Заметим, что функции это тоже объекты. Следовательно, можем прописать им _```deriving```_, правда, что он нам определит, не совсем понятно. Поэтому так делать не стоит:

```
data FunBox = FB (Int -> String)  -- remember? functions are first class values
    deriving (Eq, Ord, Enum, Bounded, Show, Read, Ix) -- what can we derive?
```

Для обёрток над типами _```newtype```_ мы тоже можем задавать _```deriving```_, __даже__ если они были определены для исходного типа:

```
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

newtype Size = Size Int
    deriving (Show, Read, Eq, Ord, Num)
```